# Using `FEniCS`

In this demo we will show how to generate Purkinje networks on a patient specific mesh from the paper [Efficient estimation of personalized
biventricular mechanical function employing gradient-based optimization](https://doi.org/10.1002/cnm.2982). Instructions on how to get the data is described in more detail [here](https://bitbucket.org/finsberg/efficient-estimation-of-personalized-biventricular-mechanical/src/master/). First you need to download the data. Next we will load the data with [fenics-pulse](https://github.com/finsberg/pulse)


In [ ]:
import numpy
import logging
import pulse
import dolfin
from fractal_tree import generate_fractal_tree, FractalTreeParameters, Mesh

Set logging level and a seed for reproducibility. 

In [ ]:
logging.basicConfig(level=logging.INFO)
numpy.random.seed(12)

Let us download the data from the paper

In [ ]:
import urllib.request
import tarfile
import time
link = "https://www.dropbox.com/s/pxbx3ohix8e3jsx/data.tar?dl=1"
path = "data.tar"
print("Downloading data")
urllib.request.urlretrieve(link, path)
time.sleep(1.0)
print("Done downloading data. Extracting...")
data_file = tarfile.open(name=path, mode="r")
data_file.extractall()
print("Done extracting")

When you download the data from the paper you will get a tarball with the data. We assume here that the data is extracted into a folder mesh. Here we will use the medium mesh for case 1

In [ ]:
case = "CASE1_med"
path = f"data/mesh/{case}_60.h5"

We will load the data with `pulse`

In [ ]:
geo = pulse.HeartGeometry.from_file(path)

We would like to visualize the mesh in Paraview afterwards, so let us save the mesh in an XDMF file. 

In [ ]:
with dolfin.XDMFFile(f"{case}_mesh.xdmf") as xdmf:
    xdmf.write(geo.mesh)

To load the mesh into `fractal-tree` we need to first get the coordinates

In [ ]:
verts = geo.mesh.coordinates()

and the connectivities

In [ ]:
connectivity = []
for facet in dolfin.facets(geo.mesh):
    if geo.ffun[facet] == geo.markers["ENDO_RV"][0]:
        connectivity.append(facet.entities(0))
connectivity = numpy.array(connectivity)

Alternatively we could save the facet function to `.xdmf` and load it with `meshio`

```
ffun_file = f"{case}_ffun.xdmf"
with dolfin.XDMFFile(ffun_file) as xdmf:
    xdmf.write(geo.ffun)
msh = meshio.read("data/mesh/CASE1_med_ffun.xdmf")
marker = 20  # ENDO RV
conn = msh.cells[0].data[msh.cell_data["f"][0].squeeze() == marker, :]
verts = msh.points
```

Next we will choose the initial node. Here you could open the mesh in Paraview and find the approximate coordinate to this node, and then find the the closest node


In [ ]:
init_node = [1.0, 15.85, 15.36]
index = numpy.linalg.norm(numpy.subtract(verts, init_node), axis=1).argmin()

Now we can create the mesh for the fractal tree


In [ ]:
mesh = Mesh(verts=verts, connectivity=connectivity, init_node=verts[index, :])

We set the fractal tree parameters and choose the initial direction as pointing in the positive $x$-direction

In [ ]:
param = FractalTreeParameters(
    filename="case1-rv",
    N_it=300,
    length=0.3,
    initial_direction=numpy.array([1, 0, 0]),
)

and finally run the fractal tree algorithm


In [ ]:
branches, nodes = generate_fractal_tree(mesh, param)

```{figure} ../docs/figures/fenics.jpg
---
name: fenics.png
---
Purkinje networks on a patient specific biventricular mesh
```